In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv("stroke.csv")

In [3]:
print(df.dtypes)

id                     int64
gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object


In [4]:
df = df.dropna()

In [5]:
y = df['stroke']
x = df.drop('stroke', axis=1)

In [6]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, random_state=42)

In [7]:
symmetric_cols = ['hypertension', 'heart_disease']
skewed_cols = ['id', 'age', 'avg_glucose_level', 'bmi']
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

symmetric_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

skewed_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy='mean')),
  ('log_transform', FunctionTransformer(np.log1p)),
  ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
  ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer([
    ('numeric_symmetric', symmetric_pipeline, symmetric_cols),
    ('numeric_skewed', skewed_pipeline, skewed_cols),
    ('categorical', categorical_pipeline, categorical_cols)
])

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [9]:
pipeline.fit(df)
train_transformed = pipeline.transform(train_x)
test_transformed = pipeline.transform(test_x)

In [10]:
linear = LinearRegression()
linear.fit(train_transformed, train_y)

y_pred = linear.predict(test_transformed)

y_temp = []
for temp in y_pred:
  if temp > 0.5:
    y_temp.append(1)
  else:
    y_temp.append(0)

print(f"Accuracy: {accuracy_score(y_temp, test_y) * 100}%")

Accuracy: 94.86970684039088%


In [11]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=12)
model.fit(train_transformed, train_y)

model_pred = model.predict(test_transformed)
print(f"Accuracy of Logistic: {accuracy_score(model_pred, test_y) * 100}%")

ValueError: Classification metrics can't handle a mix of continuous and binary targets